In [1]:
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-10-03 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-10-03 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-10-03 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-10-03 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-10-03 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [8]:
features = ["timeline_index", "speaker"]
standardise_method = [None, None]
include_features_in_path = True
include_features_in_input = False

In [9]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [11]:
size = 5

## UMAP

In [12]:
(
    swmhau_network_umap_kfold_5,
    best_swmhau_network_umap_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_5_kfold_best_model.csv


In [13]:
swmhau_network_umap_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/888675771.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_5.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.620338   
                                                                         0.0005         0.631737   
                                                                         0.0010         0.630907   
                                                          (512, 512)     0.0001         0.622989   
                                                                         0.0005         0.631354   
                                                                         0.0010         0.638985   
           12              3         10        1          (256, 256)     0.0001         0.611622   
                                                                         0.0005         0.635281   
                                                                         0.0010         0.637771   
                                                          (512, 512)     0.0001         0.615294   
                                                                         0.0005         0.630811   
                                                                         0.0010         0.637931   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.459232   
                                                                         0.0005         0.513952   
                                                                         0.0010         0.516879   
                                                          (512, 512)     0.0001         0.483902   
                                                                         0.0005         0.514762   
                                                                         0.0010         0.515646   
           12              3         10        1          (256, 256)     0.0001         0.464573   
                                                                         0.0005         0.513274   
                                                                         0.0010         0.519739   
                                                          (512, 512)     0.0001         0.468045   
                                                                         0.0005         0.508641   
                                                                         0.0010         0.517165   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.465331   
                                                                         0.0005          0.507335   
                                                                         0.0010          0.509665   
                                                          (512, 512)     0.0001          0.484949   
                                                                         0.0005          0.507829   
                                                                         0.0010          0.511491   
           12              3         10        1          (256, 256)     0.0001          0.467773   
                                                                         0.0005          0.507121   
                                                                         0.0010          0.514334   
                                                          (512, 512)     0.0001          0.468274   
                                                                         0

In [14]:
best_swmhau_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.636590,0.518346,"[0.761535015678662, 0.4498977505112475, 0.3436...",0.510492,"[0.7872800246989812, 0.44606650446066504, 0.29...",0.532223,"[0.7374204742625795, 0.4537953795379538, 0.405...",None,0.662813,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.630077,0.513121,"[0.7556818181818181, 0.43767762890783596, 0.34...",0.504989,"[0.7823529411764706, 0.4308553157474021, 0.301...",0.526981,"[0.7307692307692307, 0.44471947194719474, 0.40...",None,0.664740,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.632375,0.509376,"[0.7595839524517088, 0.44214019079220246, 0.32...",0.502918,"[0.7811735941320294, 0.4445371142618849, 0.283...",0.521460,"[0.7391555812608445, 0.4397689768976898, 0.385...",None,0.660886,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
best_swmhau_network_umap_kfold_5["f1"].mean()

0.5136146094618682

In [16]:
best_swmhau_network_umap_kfold_5["precision"].mean()

0.5061330032703593

In [17]:
best_swmhau_network_umap_kfold_5["recall"].mean()

0.5268880834487922

In [18]:
np.stack(best_swmhau_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.7589336 , 0.44323852, 0.33867171])

In [19]:
np.stack(best_swmhau_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.78360219, 0.44048631, 0.29431051])

In [20]:
np.stack(best_swmhau_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.73578176, 0.44609461, 0.39878788])

## Random Projections

In [21]:
(
    swmhau_network_grp_kfold_5,
    best_swmhau_network_grp_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_5_kfold_best_model.csv


In [22]:
swmhau_network_grp_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/572217191.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_5.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.630332   
                                                                                      0.0005         0.636462   
                                                                                      0.0010         0.637995   
                                                                         0.2          0.0001         0.627075   
                                                                                      0.0005         0.628225   
                                                                                      0.0010         0.634291   
                                                          (512, 512)     0.1          0.0001         0.635504   
                                                                                      0.0005         0.637165   
                                                                                      0.0010         0.626756   
                                                                         0.2          0.0001         0.634738   
                                                                                      0.0005         0.634547   
                                                                                      0.0010         0.630779   
           12              3         10        1          (256, 256)     0.1          0.0001         0.633269   
                                                                                      0.0005         0.626373   
                                                                                      0.0010         0.627075   
                                                                         0.2          0.0001         0.629949   
                                                                                      0.0005         0.622797   
                                                                                      0.0010         0.637548   
                                                          (512, 512)     0.1          0.0001         0.640996   
                                                                                      0.0005         0.631737   
                                                                                      0.0010         0.632950   
                                                                         0.2          0.0001         0.633078   
                                                                                      0.0005         0.634802   
                                                                                      0.0010         0.630013   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.472205   
                                                                                      0.0005         0.522416   
                                                                                      0.0010         0.519695   
                                                                         0.2          0.0001         0.475134   
                                                                                      0.0005         0.514829   
                                                                                      0.0010         0.517799   
                                                          (512, 512)     0.1          0.0001         0.502318   
                                                                                      0.0005         0.518070   
                                

In [23]:
best_swmhau_network_grp_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.638697,0.523400,"[0.7605086013462976, 0.4555245313123255, 0.354...",0.515091,"[0.7877285404400372, 0.4409266409266409, 0.316...",0.536016,"[0.735106998264893, 0.47112211221122113, 0.401...",None,0.657514,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.633908,0.522272,"[0.757717211263364, 0.45303408146300916, 0.356...",0.513695,"[0.7904492617027961, 0.4564489112227806, 0.294...",0.542722,"[0.7275882012724117, 0.44966996699669964, 0.45...",None,0.660726,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.636782,0.521576,"[0.758166017380881, 0.462161068393363, 0.34440...",0.513185,"[0.7866915422885572, 0.45353455123113584, 0.29...",0.536071,"[0.7316367842683632, 0.47112211221122113, 0.40...",None,0.665222,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [24]:
best_swmhau_network_grp_kfold_5["f1"].mean()

0.5224161963039674

In [25]:
best_swmhau_network_grp_kfold_5["precision"].mean()

0.5139905159896844

In [26]:
best_swmhau_network_grp_kfold_5["recall"].mean()

0.5382697770451809

In [27]:
np.stack(best_swmhau_network_grp_kfold_5["f1_scores"]).mean(axis=0)

array([0.75879728, 0.45690656, 0.35154475])

In [28]:
np.stack(best_swmhau_network_grp_kfold_5["precision_scores"]).mean(axis=0)

array([0.78828978, 0.45030337, 0.3033784 ])

In [29]:
np.stack(best_swmhau_network_grp_kfold_5["recall_scores"]).mean(axis=0)

array([0.73144399, 0.4639714 , 0.41939394])

# w=11

In [30]:
size = 11

## UMAP

In [31]:
(
    swmhau_network_umap_kfold_11,
    best_swmhau_network_umap_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_11_kfold_best_model.csv


In [32]:
swmhau_network_umap_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/363273794.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.609291   
                                                                         0.0005         0.627139   
                                                                         0.0010         0.632088   
                                                          (512, 512)     0.0001         0.620466   
                                                                         0.0005         0.628831   
                                                                         0.0010         0.628831   
           12              3         10        1          (256, 256)     0.0001         0.606034   
                                                                         0.0005         0.628672   
                                                                         0.0010         0.635504   
                                                          (512, 512)     0.0001         0.602075   
                                                                         0.0005         0.623499   
                                                                         0.0010         0.635504   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.411124   
                                                                         0.0005         0.512440   
                                                                         0.0010         0.516365   
                                                          (512, 512)     0.0001         0.450424   
                                                                         0.0005         0.514609   
                                                                         0.0010         0.511065   
           12              3         10        1          (256, 256)     0.0001         0.436728   
                                                                         0.0005         0.502501   
                                                                         0.0010         0.515553   
                                                          (512, 512)     0.0001         0.454487   
                                                                         0.0005         0.501562   
                                                                         0.0010         0.511854   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.427274   
                                                                         0.0005          0.504781   
                                                                         0.0010          0.510980   
                                                          (512, 512)     0.0001          0.461201   
                                                                         0.0005          0.506775   
                                                                         0.0010          0.504997   
           12              3         10        1          (256, 256)     0.0001          0.444565   
                                                                         0.0005          0.497486   
                                                                         0.0010          0.511229   
                                                          (512, 512)     0.0001          0.455721   
                                                                         0

In [33]:
best_swmhau_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.638123,0.514376,"[0.7637378783426388, 0.4374460742018982, 0.341...",0.509476,"[0.776284348864994, 0.45840867992766726, 0.293...",0.526333,"[0.7515905147484094, 0.4183168316831683, 0.409...",None,0.674213,...,0.001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.624330,0.512319,"[0.750723313537384, 0.44920634920634916, 0.337...",0.503172,"[0.7928594403345127, 0.4327217125382263, 0.283...",0.531461,"[0.7128397917871602, 0.466996699669967, 0.4145...",None,0.664098,...,0.001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.643487,0.523173,"[0.7650593580536421, 0.44540727902946275, 0.35...",0.518076,"[0.7756315007429421, 0.46897810218978103, 0.30...",0.535379,"[0.7547715442452284, 0.4240924092409241, 0.427...",None,0.668754,...,0.001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [34]:
best_swmhau_network_umap_kfold_11["f1"].mean()

0.5166230086614184

In [35]:
best_swmhau_network_umap_kfold_11["precision"].mean()

0.5102411808014652

In [36]:
best_swmhau_network_umap_kfold_11["recall"].mean()

0.531057431364883

In [37]:
np.stack(best_swmhau_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.75984018, 0.4440199 , 0.34600894])

In [38]:
np.stack(best_swmhau_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.78159176, 0.4533695 , 0.29576228])

In [39]:
np.stack(best_swmhau_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.73973395, 0.43646865, 0.4169697 ])

## Random Projections

In [40]:
(
    swmhau_network_grp_kfold_11,
    best_swmhau_network_grp_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_11_kfold_best_model.csv


In [41]:
swmhau_network_grp_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/2867464325.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.630204   
                                                                                      0.0005         0.641188   
                                                                                      0.0010         0.634738   
                                                                         0.2          0.0001         0.636462   
                                                                                      0.0005         0.633780   
                                                                                      0.0010         0.639208   
                                                          (512, 512)     0.1          0.0001         0.638059   
                                                                                      0.0005         0.637037   
                                                                                      0.0010         0.624585   
                                                                         0.2          0.0001         0.628161   
                                                                                      0.0005         0.632503   
                                                                                      0.0010         0.628033   
           12              3         10        1          (256, 256)     0.1          0.0001         0.630524   
                                                                                      0.0005         0.623946   
                                                                                      0.0010         0.634610   
                                                                         0.2          0.0001         0.632056   
                                                                                      0.0005         0.630396   
                                                                                      0.0010         0.638570   
                                                          (512, 512)     0.1          0.0001         0.636015   
                                                                                      0.0005         0.639464   
                                                                                      0.0010         0.630013   
                                                                         0.2          0.0001         0.635057   
                                                                                      0.0005         0.634100   
                                                                                      0.0010         0.630077   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.459422   
                                                                                      0.0005         0.521608   
                                                                                      0.0010         0.516203   
                                                                         0.2          0.0001         0.459282   
                                                                                      0.0005         0.516363   
                                                                                      0.0010         0.520160   
                                                          (512, 512)     0.1          0.0001         0.487402   
                                                                                      0.0005         0.519847   
                                

In [42]:
best_swmhau_network_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.643678,0.522231,"[0.7646972152644762, 0.4540983606557376, 0.347...",0.516301,"[0.7795133673775909, 0.4511400651465798, 0.318...",0.530389,"[0.7504337767495662, 0.4570957095709571, 0.383...",None,0.667951,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.639464,0.523397,"[0.7632445375636037, 0.4560686554965263, 0.350...",0.515000,"[0.7909429280397022, 0.4518218623481781, 0.302...",0.538666,"[0.7374204742625795, 0.4603960396039604, 0.418...",None,0.668593,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.640421,0.519197,"[0.7643047731989326, 0.4517453798767967, 0.341...",0.512340,"[0.7840632603406326, 0.44971381847914965, 0.30...",0.530074,"[0.7455176402544823, 0.4537953795379538, 0.390...",None,0.671162,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [43]:
best_swmhau_network_grp_kfold_11["f1"].mean()

0.5216083105147596

In [44]:
best_swmhau_network_grp_kfold_11["precision"].mean()

0.5145470652452239

In [45]:
best_swmhau_network_grp_kfold_11["recall"].mean()

0.5330429214118636

In [46]:
np.stack(best_swmhau_network_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.76408218, 0.4539708 , 0.34677196])

In [47]:
np.stack(best_swmhau_network_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.78483985, 0.45089192, 0.30790943])

In [48]:
np.stack(best_swmhau_network_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.7444573 , 0.45709571, 0.39757576])

# w=20

In [49]:
size = 20

## UMAP

In [50]:
(
    swmhau_network_umap_kfold_20,
    best_swmhau_network_umap_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold_best_model.csv


In [51]:
swmhau_network_umap_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/2229529687.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.603768   
                                                                         0.0005         0.632152   
                                                                         0.0010         0.638665   
                                                          (512, 512)     0.0001         0.604246   
                                                                         0.0005         0.631386   
                                                                         0.0010         0.641635   
           12              3         10        1          (256, 256)     0.0001         0.601309   
                                                                         0.0005         0.623467   
                                                                         0.0010         0.637676   
                                                          (512, 512)     0.0001         0.603512   
                                                                         0.0005         0.623659   
                                                                         0.0010         0.640326   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.385219   
                                                                         0.0005         0.513880   
                                                                         0.0010         0.521827   
                                                          (512, 512)     0.0001         0.409683   
                                                                         0.0005         0.515716   
                                                                         0.0010         0.516632   
           12              3         10        1          (256, 256)     0.0001         0.432168   
                                                                         0.0005         0.494785   
                                                                         0.0010         0.520816   
                                                          (512, 512)     0.0001         0.449154   
                                                                         0.0005         0.502274   
                                                                         0.0010         0.517940   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.407760   
                                                                         0.0005          0.507002   
                                                                         0.0010          0.516498   
                                                          (512, 512)     0.0001          0.428929   
                                                                         0.0005          0.508258   
                                                                         0.0010          0.511895   
           12              3         10        1          (256, 256)     0.0001          0.439752   
                                                                         0.0005          0.489459   
                                                                         0.0010          0.514310   
                                                          (512, 512)     0.0001          0.453824   
                                                                         0

In [52]:
best_swmhau_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.633525,0.513819,"[0.7613636363636365, 0.44298065047344587, 0.33...",0.506264,"[0.788235294117647, 0.44207066557107644, 0.288...",0.528538,"[0.7362637362637363, 0.4438943894389439, 0.405...",None,0.676301,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.635057,0.522246,"[0.7618184564265216, 0.4489125974558884, 0.356...",0.513183,"[0.797344293392349, 0.446530612244898, 0.29567...",0.542639,"[0.7293233082706767, 0.4513201320132013, 0.447...",None,0.662653,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.641571,0.521499,"[0.7670303745207904, 0.4386422976501305, 0.358...",0.515938,"[0.7824909747292419, 0.46408839779005523, 0.30...",0.537216,"[0.7521688837478311, 0.4158415841584158, 0.443...",None,0.672126,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [53]:
best_swmhau_network_umap_kfold_20["f1"].mean()

0.519187772649011

In [54]:
best_swmhau_network_umap_kfold_20["precision"].mean()

0.5117949221364934

In [55]:
best_swmhau_network_umap_kfold_20["recall"].mean()

0.5361306300284935

In [56]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.76340416, 0.44351185, 0.35064731])

In [57]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.78935685, 0.45089656, 0.29513135])

In [58]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.73925198, 0.4370187 , 0.43212121])

## Random Projections

In [59]:
(
    swmhau_network_grp_kfold_20,
    best_swmhau_network_grp_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_20_kfold_best_model.csv


In [60]:
swmhau_network_grp_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/1955217834.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.614112   
                                                                                      0.0005         0.638186   
                                                                                      0.0010         0.635696   
                                                                         0.2          0.0001         0.623308   
                                                                                      0.0005         0.631162   
                                                                                      0.0010         0.634483   
                                                          (512, 512)     0.1          0.0001         0.630715   
                                                                                      0.0005         0.637676   
                                                                                      0.0010         0.631545   
                                                                         0.2          0.0001         0.630779   
                                                                                      0.0005         0.631290   
                                                                                      0.0010         0.632439   
           12              3         10        1          (256, 256)     0.1          0.0001         0.624521   
                                                                                      0.0005         0.622095   
                                                                                      0.0010         0.638442   
                                                                         0.2          0.0001         0.621456   
                                                                                      0.0005         0.630140   
                                                                                      0.0010         0.636845   
                                                          (512, 512)     0.1          0.0001         0.621967   
                                                                                      0.0005         0.636143   
                                                                                      0.0010         0.631992   
                                                                         0.2          0.0001         0.630268   
                                                                                      0.0005         0.629949   
                                                                                      0.0010         0.636845   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.466954   
                                                                                      0.0005         0.524098   
                                                                                      0.0010         0.515996   
                                                                         0.2          0.0001         0.471234   
                                                                                      0.0005         0.516880   
                                                                                      0.0010         0.517464   
                                                          (512, 512)     0.1          0.0001         0.495197   
                                                                                      0.0005         0.519571   
                                

In [61]:
best_swmhau_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.614559,0.474004,"[0.7519719544259422, 0.38280616382806165, 0.28...",0.472138,"[0.7597402597402597, 0.3763955342902711, 0.280...",0.476115,"[0.7443609022556391, 0.38943894389438943, 0.29...",None,0.672286,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.625287,0.477323,"[0.7587982832618025, 0.38641188959660294, 0.28...",0.478999,"[0.7508493771234428, 0.39807524059492566, 0.28...",0.475928,"[0.7669172932330827, 0.37541254125412543, 0.28...",None,0.678388,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.626054,0.502801,"[0.7543911878535278, 0.4238151194672934, 0.330...",0.497508,"[0.7773006134969325, 0.4034302759134974, 0.311...",0.510024,"[0.7327935222672065, 0.44636963696369636, 0.35...",None,0.671965,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [62]:
best_swmhau_network_grp_kfold_20["f1"].mean()

0.48470926457915864

In [63]:
best_swmhau_network_grp_kfold_20["precision"].mean()

0.48288163634738

In [64]:
best_swmhau_network_grp_kfold_20["recall"].mean()

0.48735577008635894

In [65]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.75505381, 0.39767772, 0.30139626])

In [66]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.76263008, 0.39263368, 0.29338114])

In [67]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.74802391, 0.40374037, 0.31030303])

# w=35

In [68]:
size = 35

## UMAP

In [69]:
(
    swmhau_network_umap_kfold_35,
    best_swmhau_network_umap_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_35_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_35_kfold_best_model.csv


In [70]:
swmhau_network_umap_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/548979897.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_35.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.593934   
                                                                         0.0005         0.627427   
                                                                         0.0010         0.632950   
                                                          (512, 512)     0.0001         0.599617   
                                                                         0.0005         0.632727   
                                                                         0.0010         0.632024   
           12              3         10        1          (256, 256)     0.0001         0.588985   
                                                                         0.0005         0.619891   
                                                                         0.0010         0.625798   
                                                          (512, 512)     0.0001         0.591028   
                                                                         0.0005         0.613027   
                                                                         0.0010         0.631450   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.390413   
                                                                         0.0005         0.507085   
                                                                         0.0010         0.518019   
                                                          (512, 512)     0.0001         0.415420   
                                                                         0.0005         0.511531   
                                                                         0.0010         0.513059   
           12              3         10        1          (256, 256)     0.0001         0.422396   
                                                                         0.0005         0.494118   
                                                                         0.0010         0.510472   
                                                          (512, 512)     0.0001         0.440987   
                                                                         0.0005         0.490600   
                                                                         0.0010         0.512296   

                                                                                        precision  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate              
15         8               4         6         1          (256, 256)     0.0001          0.405132   
                                                                         0.0005          0.500641   
                                                                         0.0010          0.511158   
                                                          (512, 512)     0.0001          0.427870   
                                                                         0.0005          0.504813   
                                                                         0.0010          0.507141   
           12              3         10        1          (256, 256)     0.0001          0.426169   
                                                                         0.0005          0.487869   
                                                                         0.0010          0.503684   
                                                          (512, 512)     0.0001          0.442288   
                                                                         0

In [71]:
best_swmhau_network_umap_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.615900,0.470222,"[0.7566293459045375, 0.4083640836408364, 0.245...",0.467258,"[0.7711711711711712, 0.4058679706601467, 0.224...",0.474809,"[0.7426257952573742, 0.41089108910891087, 0.27...",None,0.675979,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.620115,0.505004,"[0.7494343038165636, 0.4261224489795919, 0.339...",0.496610,"[0.783349101229896, 0.42164781906300486, 0.284...",0.523009,"[0.7183342972816658, 0.4306930693069307, 0.42]",None,0.669236,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.618966,0.501638,"[0.7480777928539123, 0.42376856919468336, 0.33...",0.493887,"[0.7814173228346457, 0.4026745913818722, 0.297...",0.514281,"[0.7174667437825333, 0.4471947194719472, 0.378...",None,0.665703,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [72]:
best_swmhau_network_umap_kfold_35["f1"].mean()

0.4922879075088247

In [73]:
best_swmhau_network_umap_kfold_35["precision"].mean()

0.48591839085193883

In [74]:
best_swmhau_network_umap_kfold_35["recall"].mean()

0.5040329581444746

In [75]:
np.stack(best_swmhau_network_umap_kfold_35["f1_scores"]).mean(axis=0)

array([0.75138048, 0.41941837, 0.30606487])

In [76]:
np.stack(best_swmhau_network_umap_kfold_35["precision_scores"]).mean(axis=0)

array([0.77864587, 0.41006346, 0.26904585])

In [77]:
np.stack(best_swmhau_network_umap_kfold_35["recall_scores"]).mean(axis=0)

array([0.72614228, 0.42959296, 0.35636364])

## Random Projections

In [78]:
(
    swmhau_network_grp_kfold_35,
    best_swmhau_network_grp_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_35_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_grp_focal_2_35_kfold_best_model.csv


In [79]:
swmhau_network_grp_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2005997/2891164155.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_35.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.598851   
                                                                                      0.0005         0.634930   
                                                                                      0.0010         0.639847   
                                                                         0.2          0.0001         0.614049   
                                                                                      0.0005         0.625415   
                                                                                      0.0010         0.626309   
                                                          (512, 512)     0.1          0.0001         0.619860   
                                                                                      0.0005         0.629119   
                                                                                      0.0010         0.635568   
                                                                         0.2          0.0001         0.608748   
                                                                                      0.0005         0.631418   
                                                                                      0.0010         0.634738   
           12              3         10        1          (256, 256)     0.1          0.0001         0.604151   
                                                                                      0.0005         0.623499   
                                                                                      0.0010         0.639974   
                                                                         0.2          0.0001         0.609706   
                                                                                      0.0005         0.624840   
                                                                                      0.0010         0.637548   
                                                          (512, 512)     0.1          0.0001         0.609195   
                                                                                      0.0005         0.629502   
                                                                                      0.0010         0.627969   
                                                                         0.2          0.0001         0.611622   
                                                                                      0.0005         0.620243   
                                                                                      0.0010         0.635249   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.445935   
                                                                                      0.0005         0.520999   
                                                                                      0.0010         0.521835   
                                                                         0.2          0.0001         0.460678   
                                                                                      0.0005         0.512821   
                                                                                      0.0010         0.514044   
                                                          (512, 512)     0.1          0.0001         0.475734   
                                                                                      0.0005         0.515756   
                                

In [80]:
best_swmhau_network_grp_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.591379,0.466577,"[0.7259013425856087, 0.3855243722304284, 0.288...",0.465068,"[0.7587511825922422, 0.34893048128342247, 0.28...",0.471854,"[0.6957779063042221, 0.4306930693069307, 0.289...",None,0.677906,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.612644,0.467092,"[0.7468224981738496, 0.3958171959721146, 0.258...",0.470193,"[0.7546501328609388, 0.372992700729927, 0.2829...",0.466318,"[0.7391555812608445, 0.42161716171617164, 0.23...",None,0.672929,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.623563,0.492072,"[0.7555687555687556, 0.4196670538133953, 0.300...",0.488943,"[0.7765567765567766, 0.3953318745441284, 0.294...",0.496718,"[0.7356853672643147, 0.4471947194719472, 0.307...",None,0.681599,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [81]:
best_swmhau_network_grp_kfold_35["f1"].mean()

0.4752470078906279

In [82]:
best_swmhau_network_grp_kfold_35["precision"].mean()

0.474734670614678

In [83]:
best_swmhau_network_grp_kfold_35["recall"].mean()

0.4782965844299873

In [84]:
np.stack(best_swmhau_network_grp_kfold_35["f1_scores"]).mean(axis=0)

array([0.7427642 , 0.40033621, 0.28264062])

In [85]:
np.stack(best_swmhau_network_grp_kfold_35["precision_scores"]).mean(axis=0)

array([0.76331936, 0.37241835, 0.2884663 ])

In [86]:
np.stack(best_swmhau_network_grp_kfold_35["recall_scores"]).mean(axis=0)

array([0.72353962, 0.43316832, 0.27818182])

# w=80

In [87]:
size = 80

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_80,
    best_swmhau_network_umap_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_80

In [ ]:
best_swmhau_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_80,
    best_swmhau_network_grp_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_80

In [ ]:
best_swmhau_network_grp_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_80["recall_scores"]).mean(axis=0)

## w=110

In [ ]:
size = 110

## UMAP

In [ ]:
(
    swmhau_network_umap_kfold_110,
    best_swmhau_network_umap_kfold_110,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_110.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_110

In [ ]:
best_swmhau_network_umap_kfold_110["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_110["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_110["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_110["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_110["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    swmhau_network_grp_kfold_110,
    best_swmhau_network_grp_kfold_110,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_grp_kfold_110.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_grp_kfold_110

In [ ]:
best_swmhau_network_grp_kfold_110["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_110["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_110["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_110["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_110["recall_scores"]).mean(axis=0)